# VacationPy

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests, json
import gmaps
# import json
import os

# Import API key
from config import key

In [ ]:
# Load city data from WeatherPy
city_data = "../WeatherPy/Output/city_data.csv"
city_data_df = pd.read_csv(city_data, index_col=[0], encoding="ISO-8859-1")
city_data_df.head()
city_data_df.dtypes

In [ ]:
# Create new dataframe with dropped NA values 
city_loc = city_data_df.dropna(axis = 0, how ='any')
# city_data_cleandf = city_data_dropna[(city_data_dropna != 0).all(1)]

# comparing sizes of data frames
print("Old data frame length:", len(city_data_df), "\nNew data frame length:", 
       len(city_loc), "\nNumber of rows with at least 1 NA value: ",
       (len(city_data_df)-len(city_loc)))
city_loc.to_csv("./output_data/city_loc.csv")

### Humidity Heatmap

In [ ]:
# Configure gmaps
gmaps.configure(api_key=key)

# Set Locations for the heatmap
locations = city_loc[["City_Lat", "City_Long"]]

humidity = city_loc["Humidity"].astype(float)

In [ ]:
# Plot humidity heatmap
fig = gmaps.figure()

# Create humidity heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, 
                                 max_intensity=100, point_radius=3)
# Add layer and display fig
fig.add_layer(heat_layer)
fig

### Ideal Weather Conditions

In [ ]:
# Create new dataframe based on indicated parameters
ideal_cities = city_loc[(city_loc['Temperature']>70) & (city_loc['Temperature']<90) 
                   & (city_loc['Humidity']>40) & (city_loc['Humidity']<70) 
                   & (city_loc['Wind_Speed']<10) & (city_loc['Cloudiness']<30)]
ideal_cities.head(10)
# len(rslt_df)    #---- used to check number of cities that meet criteria; uncomment to enable 

### Find Nearest Hotel

In [ ]:
# PSUEDO CODE
# ForLoop: loop through dataframe
# Use try block to test for errors, if true then use except to handle errors, else
# Get latitude & longitude,and make a request to gmaps to search for first hotel within 5000 m radius
# Append data into hotel dict.
# Use timer to delay requests to not exceed query limits

# Set up dictionary of variables where data from api calls will be stored
hotel_data = {'Hotel Name':[], 'City':[], 'Country':[], 
           'City_Lat':[], 'City_Long':[], 'Date':[]
          }
# Nearby search base URL
baseurl = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Required parameters for Nearby search requests
param = {
    'keyword': 'lodging',
    'location': ,
    'radius': 5000, 
    'key': key
}

for index, row in ideal_cities.head(10).iterrows():
    try:
        


